In [1]:
from glob import glob
import numpy as np
import folium
from folium import plugins
from folium.plugins import HeatMap
import matplotlib.cm
import time
import json
import branca
import branca.colormap as cm
import pandas as pd
import geopandas as gpd
import shapely
from shapely.geometry import Point
from shapely.prepared import prep

from folium_raster import row_to_lat, lat_to_row

In [2]:
# Load predictions
df = pd.read_csv("s3://w210-poverty-mapper/modeling/metadata/source_data/meta_data_bangladesh-nepal_labeled_GM_v3.csv")

# Drop any partials
df = df.loc[df["partial_updated"] == False]

In [3]:
# Parse center lat/lon coordinates
df["lon"] = df["center"].apply(lambda row: float(row.split(", ")[0][1:]))
df["lat"] = df["center"].apply(lambda row: float(row.split(", ")[1][:-1]))

In [6]:
# Get raster layer with mercator projection
res = 100.0

data = {}

for country in df["countries"].unique():
    print(country)
    df_country = df.loc[df["countries"] == country]

    data[country] = {}
    
    data[country]["bounds"] = {}
    data[country]["bounds"]["min_lat"] = min(df_country["lat"])
    data[country]["bounds"]["max_lat"] = max(df_country["lat"])
    data[country]["bounds"]["min_lon"] = min(df_country["lon"])
    data[country]["bounds"]["max_lon"] = max(df_country["lon"])
    
    data[country]["prediction_raster"] = -1 + np.zeros((int((data[country]["bounds"]["max_lat"] - data[country]["bounds"]["min_lat"]) * res), int((data[country]["bounds"]["max_lon"] - data[country]["bounds"]["min_lon"]) * res)))
    data[country]["label_raster"] = -1 + np.zeros((int((data[country]["bounds"]["max_lat"] - data[country]["bounds"]["min_lat"]) * res), int((data[country]["bounds"]["max_lon"] - data[country]["bounds"]["min_lon"]) * res)))

    print(data[country]["prediction_raster"].shape)

    # Determine pixel to latitude transformation
    a = 1.0
    b = -lat_to_row(data[country]["bounds"]["min_lat"], 1.0, 0.0)
    a = (data[country]["prediction_raster"].shape[0] + 1) / lat_to_row(data[country]["bounds"]["max_lat"], a, b)
    
    tic = time.time()
    for ii in range(data[country]["prediction_raster"].shape[0]): 
        toc = time.time()
        if ii > 0: 
            est = ((toc - tic) / ii)
            print("Remaining {}h".format(est * (data[country]["prediction_raster"].shape[0] - ii) / 3600.0))
            
        lat = row_to_lat(ii + 0.5, a, b)

        for jj in range(data[country]["prediction_raster"].shape[1]): 
            lon = data[country]["bounds"]["min_lon"] + (jj + 0.5) / res
            
            # Great circle distance approximation: https://www.movable-type.co.uk/scripts/latlong.html
            diff = (df[["lat", "lon"]] - [lat, lon])
            diff["lon"] = diff["lon"].multiply(np.cos(2 * np.pi * (0.5 * (df["lat"] + lat) / 360.0)))
            
            # Each degree latitude has 111km
            dist2 = 111.0**2 * diff.pow(2).sum(axis=1)
            min_ind = dist2.idxmin()
            min_dist = np.sqrt(dist2[min_ind])
            
            if min_dist > 5.0: 
                data[country]["prediction_raster"][ii, jj] = -1 # Transparent
                data[country]["label_raster"][ii, jj] = -1 # Transparent

            else: 
                if np.isnan(df["prediction"][min_ind]): 
                    data[country]["prediction_raster"][ii, jj] = -2 # Grey 
                else: 
                    data[country]["prediction_raster"][ii, jj] = df["prediction"][min_ind] # Set data
                    
                if np.isnan(df["label"][min_ind]): 
                    data[country]["label_raster"][ii, jj] = -2 # Grey 
                else: 
                    data[country]["label_raster"][ii, jj] = df["label"][min_ind] # Set data

    import ujson 
    with open("map_data_mercator_resolution_{}.json".format(res), "w") as f: 
         f.write(ujson.dumps(data))       
    

['Tajikistan']
(437, 781)
Remaining 0.9500737905502319h
Remaining 0.9404899388551712h
Remaining 0.9190237697407051h
Remaining 0.9043487662573656h
Remaining 0.8812734203338624h
Remaining 0.8606578334282945h
Remaining 0.8456951109189836h
Remaining 0.834593106235067h
Remaining 0.824546331623454h
Remaining 0.8173102754486932h
Remaining 0.8104505728230332h
Remaining 0.8066593834923373h
Remaining 0.8011437691582574h
Remaining 0.7997682473489216h
Remaining 0.7956830246095304h
Remaining 0.7933609352798926h
Remaining 0.7889734525306552h
Remaining 0.7856473921220979h
Remaining 0.7808861626519097h
Remaining 0.7753931103249391h
Remaining 0.7703028583022021h
Remaining 0.7657185331137493h
Remaining 0.7611521553993225h
Remaining 0.758157345296608h
Remaining 0.7602486819267273h
Remaining 0.7587495760199352h
Remaining 0.7552993764357312h
Remaining 0.75224188109712h
Remaining 0.7483673170243186h
Remaining 0.7446838980493722h
Remaining 0.7411202718493759h
Remaining 0.7376310314051807h
Remaining 0.7342600